# HW-7

## Part-1

In [12]:
!curl -L -o cifake.zip https://www.kaggle.com/api/v1/datasets/download/birdy654/cifake-real-and-ai-generated-synthetic-images

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  104M  100  104M    0     0  87.9M      0  0:00:01  0:00:01 --:--:--  132M


In [13]:
import zipfile
zipfile.ZipFile("cifake.zip", "r").extractall("cifake")

In [14]:
import os, shutil, random

base_dir = "cifake/train"
val_dir = "cifake/validation"
os.makedirs(f"{val_dir}/REAL", exist_ok=True)
os.makedirs(f"{val_dir}/FAKE", exist_ok=True)

for category in ['REAL', 'FAKE']:
    src_dir = os.path.join(base_dir, category)
    dest_dir = os.path.join(val_dir, category)
    files = os.listdir(src_dir)
    random.shuffle(files)
    for f in files[:10000]:
        shutil.move(os.path.join(src_dir, f), os.path.join(dest_dir, f))


## Part-2

In [15]:
import tensorflow as tf

train_ds = tf.keras.utils.image_dataset_from_directory(
    "cifake/train", image_size=(32, 32), batch_size=64)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "cifake/validation", image_size=(32, 32), batch_size=64)

test_ds = tf.keras.utils.image_dataset_from_directory(
    "cifake/test", image_size=(32, 32), batch_size=64)

Found 80000 files belonging to 2 classes.
Found 35989 files belonging to 2 classes.
Found 20000 files belonging to 2 classes.


In [16]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Rescaling(1./255, input_shape=(32, 32, 3)),

    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [17]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_1 (Rescaling)         │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 13, 13, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 6, 6, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 84,001 (328.13 KB)

 Trainable params: 84,001 (328.13 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint_cb = ModelCheckpoint(
    'best_model.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

earlystop_cb = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

In [20]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
    callbacks=[checkpoint_cb, earlystop_cb]
)


Epoch 1/30
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.7733 - loss: 0.4578
Epoch 1: val_loss improved from inf to 0.27233, saving model to best_model.h5


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 108s 85ms/step - accuracy: 0.7733 - loss: 0.4578 - val_accuracy: 0.8875 - val_loss: 0.2723
Epoch 2/30
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.8925 - loss: 0.2611
Epoch 2: val_loss improved from 0.27233 to 0.22337, saving model to best_model.h5


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 102s 81ms/step - accuracy: 0.8925 - loss: 0.2611 - val_accuracy: 0.9084 - val_loss: 0.2234
Epoch 3/30
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.9114 - loss: 0.2174
Epoch 3: val_loss improved from 0.22337 to 0.19402, saving model to best_model.h5


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 147s 85ms/step - accuracy: 0.9114 - loss: 0.2174 - val_accuracy: 0.9239 - val_loss: 0.1940
Epoch 4/30
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.9220 - loss: 0.1950
Epoch 4: val_loss improved from 0.19402 to 0.17736, saving model to best_model.h5


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 106s 85ms/step - accuracy: 0.9220 - loss: 0.1950 - val_accuracy: 0.9301 - val_loss: 0.1774
Epoch 5/30
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.9294 - loss: 0.1755
Epoch 5: val_loss improved from 0.17736 to 0.16598, saving model to best_model.h5


...

1250/1250 ━━━━━━━━━━━━━━━━━━━━ 128s 79ms/step - accuracy: 0.9357 - loss: 0.1635 - val_accuracy: 0.9370 - val_loss: 0.1611
Epoch 7/30
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.9392 - loss: 0.1522
Epoch 7: val_loss did not improve from 0.16115
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 106s 85ms/step - accuracy: 0.9392 - loss: 0.1522 - val_accuracy: 0.9263 - val_loss: 0.1875
Epoch 8/30
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.9431 - loss: 0.1470
Epoch 8: val_loss did not improve from 0.16115
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 99s 79ms/step - accuracy: 0.9431 - loss: 0.1470 - val_accuracy: 0.9346 - val_loss: 0.1667
Epoch 9/30
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.9469 - loss: 0.1351
Epoch 9: val_loss improved from 0.16115 to 0.15338, saving model to best_model.h5


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 142s 80ms/step - accuracy: 0.9469 - loss: 0.1351 - val_accuracy: 0.9403 - val_loss: 0.1534
Epoch 10/30
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.9505 - loss: 0.1264
Epoch 10: val_loss improved from 0.15338 to 0.15337, saving model to best_model.h5


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 100s 80ms/step - accuracy: 0.9505 - loss: 0.1264 - val_accuracy: 0.9410 - val_loss: 0.1534
Epoch 11/30
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.9545 - loss: 0.1180
Epoch 11: val_loss improved from 0.15337 to 0.14275, saving model to best_model.h5


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 101s 81ms/step - accuracy: 0.9545 - loss: 0.1180 - val_accuracy: 0.9459 - val_loss: 0.1428
Epoch 12/30
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.9590 - loss: 0.1064
Epoch 12: val_loss did not improve from 0.14275
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 108s 86ms/step - accuracy: 0.9590 - loss: 0.1064 - val_accuracy: 0.9432 - val_loss: 0.1504
Epoch 13/30
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.9619 - loss: 0.1001
Epoch 13: val_loss did not improve from 0.14275
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 134s 80ms/step - accuracy: 0.9619 - loss: 0.1001 - val_accuracy: 0.9449 - val_loss: 0.1479
Epoch 14/30
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.9621 - loss: 0.0954
Epoch 14: val_loss improved from 0.14275 to 0.13500, saving model to best_model.h5


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 142s 80ms/step - accuracy: 0.9621 - loss: 0.0954 - val_accuracy: 0.9510 - val_loss: 0.1350
Epoch 15/30
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.9675 - loss: 0.0851
Epoch 15: val_loss did not improve from 0.13500
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 106s 85ms/step - accuracy: 0.9675 - loss: 0.0851 - val_accuracy: 0.9493 - val_loss: 0.1429
Epoch 16/30
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.9695 - loss: 0.0789
Epoch 16: val_loss did not improve from 0.13500
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 107s 86ms/step - accuracy: 0.9695 - loss: 0.0789 - val_accuracy: 0.9462 - val_loss: 0.1648
Epoch 17/30
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.9713 - loss: 0.0762
Epoch 17: val_loss did not improve from 0.13500
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 141s 85ms/step - accuracy: 0.9713 - loss: 0.0762 - val_accuracy: 0.9487 - val_loss: 0.1576
Epoch 18/30
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.9767 - loss: 0.0630
Epoch 18

In [22]:
test_loss, test_acc = model.evaluate(test_ds)
print(f"\nTest Accuracy: {test_acc * 100:.2f}%")

313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 27ms/step - accuracy: 0.9341 - loss: 0.1857

Test Accuracy: 93.58%
